In [ ]:

# ------------------------------------------------------------
# pyshp compatibility patch (ArcGIS 2.4.2 ↔ pyshp 3.x)
# Ensures shapefile.Writer.field accepts both fieldType and field_type.
# ------------------------------------------------------------
import shapefile

if not hasattr(shapefile.Writer, "_field_original"):
    shapefile.Writer._field_original = shapefile.Writer.field

def _field_compat(self, *args, **kwargs):
    if "fieldType" in kwargs and "field_type" not in kwargs:
        kwargs["field_type"] = kwargs.pop("fieldType")
    return shapefile.Writer._field_original(self, *args, **kwargs)

shapefile.Writer.field = _field_compat

# ============================================================
# California RSV Bio-Forecast (2026–2027) Pipeline
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from Bio import Entrez, SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import random
import warnings
import os
import requests


warnings.filterwarnings("ignore")

# ------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------
ARCGIS_USERNAME = "YOUR_USER_NAME"  # ArcGIS Online username
CDC_APP_TOKEN = "YOUR_CDC_APP_TOKEN"  # CDC Socrata App Token (ensure this is valid)

def socrata_get(url, params=None, timeout=30):
    """Helper to perform GET requests to Socrata APIs with an optional App Token."""
    headers = {}
    if CDC_APP_TOKEN:
        headers["X-App-Token"] = CDC_APP_TOKEN
    r = requests.get(url, params=params or {}, headers=headers, timeout=timeout)
    r.raise_for_status()
    return r.json()

print("CDC App Token set:", bool(CDC_APP_TOKEN))

# ------------------------------------------------------------
# 1) SETUP: California Cities and County Mapping
# ------------------------------------------------------------
cities = [
    # North Coast & Mountains
    {"City": "Crescent City",    "Lat": 41.75, "Lon": -124.20, "Density": 2, "Zone": "North Coast",    "Vulnerability": 1.3},
    {"City": "Eureka",           "Lat": 40.80, "Lon": -124.16, "Density": 4, "Zone": "North Coast",    "Vulnerability": 1.2},
    {"City": "Arcata",           "Lat": 40.86, "Lon": -124.08, "Density": 4, "Zone": "North Coast",    "Vulnerability": 0.9},
    {"City": "Truckee",          "Lat": 39.33, "Lon": -120.18, "Density": 2, "Zone": "Sierra Nevada",  "Vulnerability": 0.9},
    {"City": "South Lake Tahoe","Lat": 38.94, "Lon": -119.98, "Density": 3, "Zone": "Sierra Nevada",  "Vulnerability": 1.0},
    {"City": "Mammoth Lakes",    "Lat": 37.65, "Lon": -118.97, "Density": 2, "Zone": "Sierra Nevada",  "Vulnerability": 0.9},

    # Sacramento Valley
    {"City": "Redding",    "Lat": 40.59, "Lon": -122.39, "Density": 3, "Zone": "North Valley",   "Vulnerability": 1.4},
    {"City": "Chico",      "Lat": 39.73, "Lon": -121.84, "Density": 4, "Zone": "North Valley",   "Vulnerability": 1.0},
    {"City": "Sacramento", "Lat": 38.58, "Lon": -121.49, "Density": 7, "Zone": "Central Valley", "Vulnerability": 1.0},
    {"City": "Davis",      "Lat": 38.54, "Lon": -121.74, "Density": 5, "Zone": "Central Valley", "Vulnerability": 0.7},

    # Bay Area & North Bay
    {"City": "San Francisco", "Lat": 37.77, "Lon": -122.42, "Density": 10, "Zone": "Bay Area",  "Vulnerability": 0.9},
    {"City": "Oakland",       "Lat": 37.80, "Lon": -122.27, "Density": 9,  "Zone": "Bay Area",  "Vulnerability": 1.1},
    {"City": "San Jose",      "Lat": 37.34, "Lon": -121.89, "Density": 8,  "Zone": "Bay Area",  "Vulnerability": 0.9},
    {"City": "Santa Rosa",    "Lat": 38.44, "Lon": -122.71,"Density": 5,  "Zone": "North Bay", "Vulnerability": 1.1},
    {"City": "Napa",          "Lat": 38.30, "Lon": -122.29,"Density": 4,  "Zone": "North Bay", "Vulnerability": 1.2},

    # San Joaquin Valley
    {"City": "Stockton",    "Lat": 37.96, "Lon": -121.29, "Density": 6, "Zone": "Central Valley", "Vulnerability": 1.3},
    {"City": "Fresno",      "Lat": 36.74, "Lon": -119.79, "Density": 6, "Zone": "Central Valley", "Vulnerability": 1.4},
    {"City": "Visalia",     "Lat": 36.33, "Lon": -119.29, "Density": 5, "Zone": "Central Valley", "Vulnerability": 1.3},
    {"City": "Bakersfield", "Lat": 35.37, "Lon": -119.02, "Density": 6, "Zone": "Central Valley", "Vulnerability": 1.4},

    # Central Coast
    {"City": "Monterey",        "Lat": 36.60, "Lon": -121.89, "Density": 5, "Zone": "Central Coast", "Vulnerability": 1.1},
    {"City": "San Luis Obispo","Lat": 35.28, "Lon": -120.66, "Density": 4, "Zone": "Central Coast", "Vulnerability": 0.9},
    {"City": "Santa Barbara",   "Lat": 34.42, "Lon": -119.70, "Density": 6, "Zone": "Central Coast", "Vulnerability": 1.1},
    {"City": "Santa Maria",     "Lat": 34.95, "Lon": -120.44, "Density": 5, "Zone": "Central Coast", "Vulnerability": 1.2},

    # Southern California (LA Basin, Orange County, San Diego)
    {"City": "Los Angeles",  "Lat": 34.05, "Lon": -118.24, "Density": 9, "Zone": "LA Basin",      "Vulnerability": 1.0},
    {"City": "Long Beach",   "Lat": 33.77, "Lon": -118.19, "Density": 9, "Zone": "LA Basin",      "Vulnerability": 1.0},
    {"City": "Santa Monica", "Lat": 34.02, "Lon": -118.49, "Density": 9, "Zone": "LA Basin",      "Vulnerability": 0.9},
    {"City": "Pasadena",     "Lat": 34.15, "Lon": -118.14, "Density": 8, "Zone": "LA Basin",      "Vulnerability": 1.0},
    {"City": "Lancaster",    "Lat": 34.69, "Lon": -118.15, "Density": 5, "Zone": "High Desert/LA","Vulnerability": 1.1},
    {"City": "Anaheim",      "Lat": 33.84, "Lon": -117.91, "Density": 8, "Zone": "Orange County", "Vulnerability": 1.0},
    {"City": "Irvine",       "Lat": 33.68, "Lon": -117.83, "Density": 7, "Zone": "Orange County", "Vulnerability": 0.8},
    {"City": "San Diego",    "Lat": 32.72, "Lon": -117.16, "Density": 8, "Zone": "San Diego",     "Vulnerability": 0.9},
    {"City": "Oceanside",    "Lat": 33.20, "Lon": -117.38, "Density": 6, "Zone": "San Diego",     "Vulnerability": 1.1},
    {"City": "Chula Vista",  "Lat": 32.64, "Lon": -117.08, "Density": 7, "Zone": "San Diego",     "Vulnerability": 1.0},

    # Inland Empire & Deserts
    {"City": "Riverside",      "Lat": 33.98, "Lon": -117.38, "Density": 7, "Zone": "Inland Empire","Vulnerability": 1.1},
    {"City": "San Bernardino", "Lat": 34.11, "Lon": -117.29, "Density": 7, "Zone": "Inland Empire","Vulnerability": 1.2},
    {"City": "Palm Springs",   "Lat": 33.83, "Lon": -116.55, "Density": 4, "Zone": "Low Desert",   "Vulnerability": 1.8},
    {"City": "Indio",          "Lat": 33.72, "Lon": -116.22, "Density": 4, "Zone": "Low Desert",   "Vulnerability": 1.4},
    {"City": "Barstow",        "Lat": 34.90, "Lon": -117.02, "Density": 3, "Zone": "High Desert",  "Vulnerability": 1.3},
    {"City": "Victorville",    "Lat": 34.54, "Lon": -117.29, "Density": 5, "Zone": "High Desert",  "Vulnerability": 1.2},
    {"City": "El Centro",      "Lat": 32.79, "Lon": -115.56, "Density": 4, "Zone": "Imperial Valley","Vulnerability": 1.3},
    {"City": "Needles",        "Lat": 34.85, "Lon": -114.61, "Density": 2, "Zone": "Mojave Desert", "Vulnerability": 1.4},
    {"City": "Big Bear Lake",  "Lat": 34.24, "Lon": -116.91, "Density": 2, "Zone": "Mountain",      "Vulnerability": 0.9},
]
N_CITIES = len(cities)

# Mapping from City to County (for joining county-level metrics)
CITY_TO_COUNTY_CA = {
    "Crescent City": "Del Norte",
    "Eureka": "Humboldt",
    "Arcata": "Humboldt",
    "Truckee": "Nevada",
    "South Lake Tahoe": "El Dorado",
    "Mammoth Lakes": "Mono",
    "Redding": "Shasta",
    "Chico": "Butte",
    "Sacramento": "Sacramento",
    "Davis": "Yolo",
    "San Francisco": "San Francisco",
    "Oakland": "Alameda",
    "San Jose": "Santa Clara",
    "Santa Rosa": "Sonoma",
    "Napa": "Napa",
    "Stockton": "San Joaquin",
    "Fresno": "Fresno",
    "Visalia": "Tulare",
    "Bakersfield": "Kern",
    "Monterey": "Monterey",
    "San Luis Obispo": "San Luis Obispo",
    "Santa Barbara": "Santa Barbara",
    "Santa Maria": "Santa Barbara",
    "Los Angeles": "Los Angeles",
    "Long Beach": "Los Angeles",
    "Santa Monica": "Los Angeles",
    "Pasadena": "Los Angeles",
    "Lancaster": "Los Angeles",
    "Anaheim": "Orange",
    "Irvine": "Orange",
    "San Diego": "San Diego",
    "Oceanside": "San Diego",
    "Chula Vista": "San Diego",
    "Riverside": "Riverside",
    "San Bernardino": "San Bernardino",
    "Palm Springs": "Riverside",
    "Indio": "Riverside",
    "Barstow": "San Bernardino",
    "Victorville": "San Bernardino",
    "El Centro": "Imperial",
    "Needles": "San Bernardino",
    "Big Bear Lake": "San Bernardino",
}

# ------------------------------------------------------------
# 2) DATA FETCHERS: CDC PLACES (smoking) & COVID Vaccination (proxy for RSV vaccine acceptance)
# ------------------------------------------------------------
PLACES_COUNTY_API = "https://data.cdc.gov/resource/swc5-untb.json"  # CDC PLACES county-level data


def fetch_county_smoking_rates_ca(measureid="CSMOKING", data_value_type="Age-adjusted prevalence"):
    url = "https://data.cdc.gov/resource/swc5-untb.json"
    params = {
        "stateabbr": "CA",
        "measureid": measureid,
        "data_value_type": data_value_type,
        "$select": "locationname, data_value, data_value_type, year",
        "$limit": 5000
    }
    headers = {"X-App-Token": CDC_APP_TOKEN}
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    rows = response.json()
    print(f"Retrieved {len(rows)} smoking records.")
    latest = {}
    for r in rows:
        county = r.get("locationname")
        val = r.get("data_value")
        year = r.get("year")
        if county and val and year:
            try:
                y = int(str(year))
                v = float(val)
                prev = latest.get(county)
                if prev is None or y >= prev[1]:
                    latest[county] = (v, y)
            except:
                continue
    return {k: v[0] for k, v in latest.items()}


OVID_COUNTY_API = "https://data.cdc.gov/resource/8xkx-amqh.json"  # correct endpoint

def fetch_county_vax_rate_covid_ca(
    metric="series_complete_pop_pct",
    app_token=None,
    limit=20000
):
    """
    Notes:
    - CDC county strings often look like "Los Angeles County" -> we normalize to "Los Angeles"
    - We skip 'Unknown', 'Out of State', etc.
    """
    headers = {}
    if app_token:
        headers["X-App-Token"] = app_token

    params = {
        # Filter to CA and drop junk counties
        "$where": (
            "recip_state='CA' "
            "AND recip_county IS NOT NULL "
            "AND recip_county NOT IN ('Unknown','Out of State','Federal Entities','Tribal Entity','Long Term Care')"
        ),
        "$select": f"recip_county, date, {metric}",
        "$order": "date DESC",
        "$limit": limit
    }

    r = requests.get(COVID_COUNTY_API, params=params, headers=headers, timeout=60)
    r.raise_for_status()
    rows = r.json()

    latest = {}
    for rec in rows:
        county_raw = rec.get("recip_county")
        pct = rec.get(metric)
        dt = rec.get("date")

        if not county_raw or pct in (None, "", "NA") or not dt:
            continue

        # normalize county name so it matches typical CA county naming
        county = county_raw.replace(" County", "").strip()

        try:
            v = float(pct)
        except:
            continue

        # because we sorted DESC by date, the first non-null per county is the most recent usable
        if county not in latest:
            latest[county] = v

    print(f"Retrieved {len(rows)} rows; captured {len(latest)} CA counties with non-null {metric}.")
    return latest

def fetch_state_rsv_coverage_ca():
    """
    Placeholder for fetching state-level RSV vaccination coverage (e.g., older adults 60+ in California).
    Currently returns np.nan (no data available).
    """
    return np.nan

# ------------------------------------------------------------
# 2b) BIOLOGY MODULE: RSV Fusion Protein Instability
# ------------------------------------------------------------
def generate_synthetic_proteins(n):
    """Generate n synthetic protein instability values if real data is unavailable."""
    return [random.uniform(30, 55) for _ in range(n)]

def get_real_rsv_instability(n=N_CITIES):
    """
    Fetches RSV Fusion (F) protein sequences from NCBI and computes their instability indices.
    If unable to fetch or insufficient sequences, falls back to synthetic values.
    """
    Entrez.email = "YOUR_NCBI_EMAIL"
    search_term = "Respiratory syncytial virus fusion protein"
    print("--- Contacting NCBI for RSV F protein data ---")
    try:
        handle = Entrez.esearch(db="protein", term=search_term, retmax=max(20, n))
        record = Entrez.read(handle)
        handle.close()
        if "IdList" not in record or not record["IdList"]:
            print("(!) No sequences found on NCBI. Using synthetic protein data.")
            return generate_synthetic_proteins(n)
        handle = Entrez.efetch(db="protein", id=record["IdList"], rettype="fasta", retmode="text")
        sequences = [str(r.seq) for r in SeqIO.parse(handle, "fasta")]
        handle.close()
        scores = []
        for seq in sequences:
            # Remove any non-standard amino acids
            clean_seq = "".join([aa for aa in seq if aa in "ACDEFGHIKLMNPQRSTVWY"])
            if len(clean_seq) > 200:
                analysis = ProteinAnalysis(clean_seq)
                scores.append(analysis.instability_index())
        if not scores:
            print("(!) No valid sequences found after filtering. Using synthetic protein data.")
            return generate_synthetic_proteins(n)
        # If less than n scores, repeat some to reach n
        while len(scores) < n:
            scores.append(random.choice(scores))
        print(f"-> Successfully fetched {len(scores)} instability scores from NCBI.")
        return scores[:n]
    except Exception as e:
        print(f"(!) NCBI API Error: {e}. Using synthetic protein data.")
        return generate_synthetic_proteins(n)

# ------------------------------------------------------------
# 3) TRAINING DATA GENERATOR (Synthetic historical data 2020-2025)
# ------------------------------------------------------------
def get_weather_sim(lat, month, zone):
    """
    Generate a simulated temperature (Temp_C) and rainfall (Rain_mm) for a given location and month.
    Incorporates regional differences (e.g., desert zones have larger temperature swings, less rain).
    """
    # Base temperature: 22°C at lat 32, decreases by 0.6°C per degree latitude increase
    base_temp = 22 - (0.6 * (lat - 32))
    # Temperature seasonal swing: bigger swing in desert regions
    swing = 12 if "Desert" in zone else 6
    # Seasonal temperature variation (cosine wave)
    temp = base_temp - (swing * np.cos((month - 1) * 2 * np.pi / 12))
    # Rainfall simulation: more rain in winter months, especially at higher latitudes
    if month in [11, 12, 1, 2, 3]:  # Nov through Mar as wet season
        rain = np.random.exponential(50) if lat > 35 else np.random.exponential(20)
    else:
        rain = np.random.exponential(2)
    # Deserts get a fraction of the rain
    if "Desert" in zone:
        rain *= 0.2
    return round(temp, 1), round(rain, 1)

def generate_history(protein_data):
    """
    Generates a synthetic training dataset (2020-2025) by combining:
    - Simulated weather (Temp, Rain)
    - Viral protein stability (Protein_Instability)
    - City demographics (Density, Vulnerability)
    - County health behaviors (Smoking_Rate)
    - Vaccine acceptance (Vaccination_Rate)
    Outputs a pandas DataFrame with monthly records for each city.
    """
    records = []
    # Map city index to protein instability for clarity
    city_instability = {i: protein_data[i] for i in range(N_CITIES)}
    # Weights for effects of smoking and vaccination on R0/cases
    W_SMOKING_R0 = 0.006   # R0 increases by 0.006 per 1% increase in smoking prevalence
    W_VAX_CASES  = 0.003   # Cases reduced by 0.3% per 1% increase in vaccination (proxy)
    for year in range(2020, 2025):
        for month in range(1, 13):
            for i, city in enumerate(cities):
                temp, rain = get_weather_sim(city["Lat"], month, city["Zone"])
                instability = city_instability[i]
                # Base R0 for RSV (starting around 1.1)
                r0 = 1.1
                # Environmental adjustments to R0
                if temp < 10:        # cold temperatures boost transmission
                    r0 += 0.9
                if rain > 30:        # heavy rain -> more indoor crowding
                    r0 += 0.5
                if month in [12, 1]: # holiday/winter peak months 
                    r0 += 0.6
                # Viral instability effect
                if instability < 40:    # stable virus -> higher R0
                    r0 += 0.4
                elif instability > 50:  # very unstable virus -> slightly lower R0
                    r0 -= 0.2
                # Population density effect (more density => higher R0)
                r0 += (city["Density"] * 0.15)
                # Smoking prevalence effect
                s_rate = city.get("Smoking_Rate", np.nan)
                if s_rate is not np.nan and not np.isnan(s_rate):
                    r0 += s_rate * W_SMOKING_R0
                # Compute case incidence (cases per some fixed population, scaled by vulnerability)
                cases = (r0 * 80) * city["Vulnerability"]
                # Vaccination effect on cases
                v_rate = city.get("Vaccination_Rate", np.nan)
                if v_rate is not np.nan and not np.isnan(v_rate):
                    cases *= max(0.2, (1.0 - v_rate * W_VAX_CASES))
                # Assemble record
                records.append({
                    "Year": year,
                    "Month": month,
                    "City": city["City"],
                    "Lat": city["Lat"],
                    "Lon": city["Lon"],
                    "Zone": city["Zone"],
                    "Density": city["Density"],
                    "Vulnerability": city["Vulnerability"],
                    "Temp_C": temp,
                    "Rain_mm": rain,
                    "Protein_Instability": instability,
                    "Smoking_Rate": s_rate,
                    "Vaccination_Rate": v_rate,
                    "Cases": cases
                })
    return pd.DataFrame(records)

# ------------------------------------------------------------
# 4) MAIN EXECUTION: Data Fetching, Model Training, Forecasting
# ------------------------------------------------------------
USE_VAX_PROXY = True  # If True, use COVID vax rates as proxy; if False, use a fixed RSV coverage (state level).

print("Fetching RSV F-protein instability data...")
protein_data = get_real_rsv_instability(N_CITIES)

print("Fetching county-level smoking rates and vaccination coverage...")
county_smoking = fetch_county_smoking_rates_ca()
county_vax_cov = fetch_county_vax_rate_covid_ca()
state_rsv_75plus = fetch_state_rsv_coverage_ca()

# Debug: verify fetched data
print("Smoking data sample:", list(county_smoking.items())[:5])
print("Vaccination data sample:", list(county_vax_cov.items())[:5])

# Enrich city data with county-level metrics
def get_city_enrichment(city_name):
    county = CITY_TO_COUNTY_CA.get(city_name)
    s_rate = county_smoking.get(county, np.nan)
    if USE_VAX_PROXY:
        v_rate = county_vax_cov.get(county, np.nan)
    else:
        v_rate = state_rsv_75plus
    return s_rate, v_rate

for city in cities:
    s, v = get_city_enrichment(city["City"])
    city["Smoking_Rate"] = s
    city["Vaccination_Rate"] = v

print("Generating synthetic historical dataset (2020-2025)...")
df_train = generate_history(protein_data)
print(f"Training data size: {df_train.shape}")

print("Training Random Forest model...")
features = ["Lat", "Density", "Vulnerability", "Temp_C", "Rain_mm", "Month",
            "Protein_Instability", "Smoking_Rate", "Vaccination_Rate"]
target = "Cases"
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(df_train[features], df_train[target])
print("Model training complete.")

# ------------------------------------------------------------
# 5) FORECAST FUTURE (2026–2027)
# ------------------------------------------------------------
print("Forecasting 24-month period (2026-2027)...")
future_records = []
W_VAX_HOSP = 0.006  # Hospitalization reduction 0.6% per 1% vaccinated
for year in [2026, 2027]:
    for month in range(1, 13):
        for i, city in enumerate(cities):
            temp, rain = get_weather_sim(city["Lat"], month, city["Zone"])
            # Assume slight trend: increase temp by 0.5°C and slightly more rain in Jan-Feb
            temp += 0.5
            if month in [1, 2]:
                rain *= 1.1
            instability = protein_data[i]  # use the same instability profile as baseline
            s_rate = city.get("Smoking_Rate", np.nan)
            v_rate = city.get("Vaccination_Rate", np.nan)
            # Predict cases using trained model
            X = np.array([[city["Lat"], city["Density"], city["Vulnerability"], temp, rain, month,
                           instability, s_rate, v_rate]])
            pred_cases = float(model.predict(X)[0])
            # Estimate hospitalization rate (per 100k) from cases
            hosp_rate = pred_cases * 0.025  # assume fixed hospitalization ratio
            if v_rate is not np.nan and not np.isnan(v_rate):
                hosp_rate *= max(0.15, (1.0 - v_rate * W_VAX_HOSP))
            future_records.append({
                "Date": pd.Timestamp(f"{year}-{month:02d}-01"),
                "Year": year,
                "Month": month,
                "City": city["City"],
                "Lat": city["Lat"],
                "Lon": city["Lon"],
                "Zone": city["Zone"],
                "Density": city["Density"],
                "Vulnerability": city["Vulnerability"],
                "Temp_C": round(temp, 1),
                "Rain_mm": round(rain, 1),
                "Protein_Instability": instability,
                "Smoking_Rate": s_rate,
                "Vaccination_Rate": v_rate,
                "Pred_Cases": int(pred_cases),
                "Hosp_Rate": round(hosp_rate, 1)
            })

df_forecast = pd.DataFrame(future_records)
print(f"Forecast data size: {df_forecast.shape}")

# ------------------------------------------------------------
# 6) EXPORT FORECAST TO CSV
# ------------------------------------------------------------
output_csv = "rsv_forecast_2026_2027.csv"
df_forecast.to_csv(output_csv, index=False)
print(f"Forecast data saved to {output_csv}")

# ------------------------------------------------------------
# 7) VISUALIZATION 1: Peaks & Valleys Graph
# ------------------------------------------------------------
print("Generating 'Peaks & Valleys' overview graph...")
plt.figure(figsize=(15, 7))
# Statewide average cases
statewide = df_forecast.groupby("Date")["Pred_Cases"].mean()
plt.plot(statewide.index, statewide.values, color="black", linewidth=3, label="Statewide Average", zorder=10)
# Plot by major zone categories (aggregated)
zones = ["Bay Area", "Central Valley", "SoCal", "Desert", "Mountain"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
for zone, color in zip(zones, colors):
    zone_mask = df_forecast["Zone"].str.contains(zone, case=False, na=False)
    zone_series = df_forecast[zone_mask].groupby("Date")["Pred_Cases"].mean()
    if not zone_series.empty:
        plt.plot(zone_series.index, zone_series.values, label=zone, color=color, alpha=0.7)
# Highlight winter periods (Jan-Feb) with a translucent red overlay
for year in [2026, 2027]:
    plt.axvspan(pd.Timestamp(f"{year}-01-01"), pd.Timestamp(f"{year}-03-01"), color="red", alpha=0.1)
plt.title("California RSV Bio-Forecast (2026-2027): Peaks & Valleys by Region", fontsize=16)
plt.ylabel("Predicted Cases per 100k population", fontsize=12)
plt.xlabel("Date", fontsize=12)
plt.legend(loc="upper right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
graph_file = "california_rsv_forecast_graph.png"
plt.savefig(graph_file, dpi=150)
plt.close()
print(f"Graph saved: {graph_file}")

# ------------------------------------------------------------
# 8) ARCGIS ONLINE MAPPING: Publish Forecast Snapshot
# ------------------------------------------------------------
print("Signing into ArcGIS Online and publishing layers...")
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import getpass

# Prompt for ArcGIS Online credentials
password = getpass.getpass(f"ArcGIS Online password for {ARCGIS_USERNAME}: ")
gis = GIS("https://www.arcgis.com", ARCGIS_USERNAME, password)
print("Signed in as:", gis.users.me.username)

# Prepare a January 2026 forecast snapshot for mapping
df_map = df_forecast[(df_forecast["Year"] == 2026) & (df_forecast["Month"] == 1)].copy()
# Ensure Date is datetime and drop unneeded columns for map
df_map["Date"] = pd.to_datetime(df_map["Date"])
# Rename columns to be shapefile/feature layer friendly (<=10 chars, no special chars)
rename_map = {
    "Vulnerability": "Vuln",
    "Protein_Instability": "ProtInst",
    "Smoking_Rate": "SmokePct",
    "Vaccination_Rate": "VaxPct",
    "Pred_Cases": "PredCases",
    "Hosp_Rate": "HospRate",
    "Density": "Density",  # <=10 chars already
    "Temp_C": "TempC",
    "Rain_mm": "RainMM"
}
for col, new_col in rename_map.items():
    if col in df_map.columns:
        df_map.rename(columns={col: new_col}, inplace=True)
# Ensure all column names <= 10 characters (just in case)
for col in list(df_map.columns):
    if len(col) > 10:
        df_map.rename(columns={col: col[:10]}, inplace=True)

# Create Spatially Enabled DataFrame
sdf = GeoAccessor.from_xy(df_map, x_column="Lon", y_column="Lat", sr=4326)

# Helper function to publish a spatial dataframe as a feature layer
def publish_sdf_as_featurelayer(sdf, gis_conn, title, tags=None, folder=None):
    tags = tags or ["RSV", "Forecast", "California", "Public Health"]
    fl_collection = sdf.spatial.to_featurelayer(title=title, gis=gis_conn, tags=tags, folder=folder)
    return fl_collection

# Publish point layer (cities with forecast data) and a heatmap layer for rainfall as examples
points_title = "California RSV Forecast (Jan 2026) - Points"
heat_title   = "California RSV Forecast (Jan 2026) - Rain Heatmap"
points_flc = publish_sdf_as_featurelayer(sdf, gis, points_title)
heat_flc   = publish_sdf_as_featurelayer(sdf, gis, heat_title)
points_layer = points_flc.layers[0]
heat_layer   = heat_flc.layers[0]

# Try to find a reference USA Counties layer for context
def find_counties_layer(gis_conn):
    queries = [
        'title:"USA Counties" AND type:"Feature Layer"', 
        'title:"USA Counties (Generalized)" AND type:"Feature Layer"',
        'title:"Counties" AND type:"Feature Layer" AND owner:esri'
    ]
    for q in queries:
        items = gis_conn.content.search(q, max_items=5)
        for item in items:
            try:
                return item.layers[0]
            except:
                continue
    return None

counties_layer = find_counties_layer(gis)

# Create a web map and add layers

webmap = gis.map("California")
webmap.zoom = 6


if counties_layer:
    # Filter counties to California only if possible
    try:
        counties_layer.definition_expression = "STATE_NAME = 'California' OR STATE_ABBR = 'CA'"
    except:
        pass
    webmap.add_layer(counties_layer)
webmap.layers = [heat_layer, points_layer]

# Configure symbology for points (based on hospitalization rate)
points_renderer = {
    "type": "classBreaks",
    "field": "HospRate",
    "classificationMethod": "esriClassifyManual",
    "minValue": 0,
    "classBreakInfos": [
        {
            "classMaxValue": 5,
            "label": "≤ 5 (Low)",
            "symbol": {
                "type": "esriSMS",
                "style": "esriSMSCircle", "size": 8,
                "color": [50, 205, 50, 180],
                "outline": {"color": [30, 30, 30, 220], "width": 1}
            }
        },
        {
            "classMaxValue": 10,
            "label": "> 5 (Medium)",
            "symbol": {
                "type": "esriSMS",
                "style": "esriSMSCircle", "size": 14,
                "color": [255, 255, 0, 180],
                "outline": {"color": [30, 30, 30, 220], "width": 1}
            }
        },
        {
            "classMaxValue": 15,
            "label": "> 10 (High)",
            "symbol": {
                "type": "esriSMS",
                "style": "esriSMSCircle", "size": 20,
                "color": [255, 140, 0, 180],
                "outline": {"color": [30, 30, 30, 220], "width": 1}
            }
        },
        {
            "classMaxValue": 9999,
            "label": "> 15 (Critical)",
            "symbol": {
                "type": "esriSMS",
                "style": "esriSMSCircle", "size": 26,
                "color": [255, 0, 0, 180],
                "outline": {"color": [30, 30, 30, 220], "width": 1}
            }
        }
    ]
}
points_popup = {
    "title": "{City} — Jan 2026 Forecast",
    "content": "<b>Hosp. Rate/100k:</b> {HospRate}<br/>"
               "<b>Pred. Cases:</b> {PredCases}<br/>"
               "<hr/>"
               "<b>Protein Instability:</b> {ProtInst}<br/>"
               "<b>Smoking (%):</b> {SmokePct}<br/>"
               "<b>Vaccination (%):</b> {VaxPct}"
}
points_layer.manager.update_definition({
    "drawingInfo": {"renderer": points_renderer},
    "popupInfo": points_popup
})

# Configure heatmap for rainfall
heat_renderer = {
    "type": "heatmap",
    "field": "RainMM",
    "blurRadius": 18,
    "maxPixelIntensity": 100,
    "minPixelIntensity": 0
}
heat_layer.manager.update_definition({
    "drawingInfo": {"renderer": heat_renderer}
})

print("ArcGIS Feature Layers published:")
# Display the map if running in an interactive environment
try:
    display(webmap)
except:
    print("Map ready. Open in Jupyter environment to view interactive map.")


CDC App Token set: True
Fetching RSV F-protein instability data...
--- Contacting NCBI for RSV F protein data ---
-> Successfully fetched 42 instability scores from NCBI.
Fetching county-level smoking rates and vaccination coverage...
Retrieved 58 smoking records.
Retrieved 20000 rows; captured 59 CA counties with non-null series_complete_pop_pct.
Smoking data sample: [('Colusa', 13.5), ('Nevada', 11.5), ('Lassen', 16.2), ('Contra Costa', 9.3), ('Imperial', 13.3)]
Vaccination data sample: [('San Mateo', 88.3), ('Tuolumne', 53.0), ('Calaveras', 45.4), ('Amador', 50.9), ('Orange', 74.7)]
Generating synthetic historical dataset (2020-2025)...
Training data size: (2520, 14)
Training Random Forest model...
Model training complete.
Forecasting 24-month period (2026-2027)...
Forecast data size: (1008, 16)
Forecast data saved to rsv_forecast_2026_2027.csv
Generating 'Peaks & Valleys' overview graph...
Graph saved: california_rsv_forecast_graph.png
Signing into ArcGIS Online and publishing laye

ArcGIS Online password for rmantilla_EBIreporting:  ········


Signed in as: rmantilla_EBIreporting
ArcGIS Feature Layers published:


Map()